In [ ]:
# Imports

import pandas as pd
from pandas import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

# Setup

In [ ]:
SPOTIFY_ONTOLOGY = "https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#"
SPARQL_ENDPOINT = "http://localhost:7200/repositories/eu"

sparql = SPARQLWrapper(SPARQL_ENDPOINT)


## Query utilities

In [ ]:
# Execute a SPARQL query and return a DataFrame
def executeQuery(query, maxRows=None):
    global sparql

    # Set the number of rows and columns to display
    pd.set_option("display.max_rows", maxRows, "display.max_columns", None)

    # Set the query
    sparql.setQuery(query)

    # Set the result format
    sparql.setReturnFormat(JSON)

    # Execute the query
    results = sparql.query().convert()

    # Create a DataFrame from the returned JSON
    resultDF = json_normalize(results["results"]["bindings"])

    # Remove the datadtypes columns
    resultDF = resultDF[[col for col in resultDF.columns if not col.endswith(
        ".type") and not col.endswith(".datatype")]]

    # Remove .value from column names
    resultDF = resultDF.rename(columns=lambda col: col.replace(".value", ""))

    # Remove the spotify ontology prefix from uris (to get only the ID)
    resultDF = resultDF.applymap(lambda x: str(x).lstrip(SPOTIFY_ONTOLOGY))

    return resultDF


# Queries

In [ ]:
executeQuery("""
    PREFIX so: <https://www.dei.unipd.it/~martinelli/spotify/spotifyOntology#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    select distinct ?track ?trackName ?danceability (group_concat(distinct(?genreName); separator=", ") as ?genresNames) (group_concat(distinct(?artistName); separator=", ") as ?artistNames) (group_concat(distinct(?artistRealCompleteName); separator=", ") as ?artistRealNames) where { 
        ?track a so:Track ;
            so:danceability ?danceability ;
            so:name ?trackName .
        ?artist a so:Artist ;
                so:partecipateIn ?track ;
                so:name ?artistName ;
                so:hasGenre ?genre .
        ?genre a so:Genre ;
            so:name ?genreName .
        OPTIONAL {
            ?artistPerson so:isMemberOf ?artist ;
                so:name ?artistRealName ;
                so:surname ?artistRealSurname .
        } .
        BIND(CONCAT(?artistRealName, " ", ?artistRealSurname) AS ?artistRealCompleteName) .
        FILTER (?danceability >= "0.9"^^xsd:float) .
    } group by ?track ?trackName ?danceability order by desc(?danceability)
""")